# Set up workspace

In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import glob
import pandas as pd
from scipy.stats import ttest_ind
from cartopy.util import add_cyclic_point
import cartopy.crs as ccrs
#from eofs.xarray import Eof 

In [2]:
import cftime
import scipy

In [3]:
import sys
sys.path.append('/glade/u/home/czarakas/coupled_PPE/code/utils')

import make_multisimulation_dataset

In [4]:
from load_ensembles_postprocessing import *
import quick_map

In [5]:
path_out='/glade/work/czarakas/coupled_PPE/data/data_for_figures/timeseries/'
#path_out='~/coupled_PPE/data/processed_data_for_figures/'

# Load data

### Define variables to load

In [6]:
var='VPD_CAN' #'TSKIN'
domain='lnd'
ensemble='offline'
season='Annual'

### Load full ensemble data

In [7]:
if ensemble=='coupled':
    ensemble_coupled = load_coupled_ensemble(var=var, domain=domain, printon=False, subdir='month_1/')
elif ensemble=='offline':
    ensemble_coupled = load_offline_ensemble(var=var, domain=domain, printon=False, subdir='month_1/')

/glade/u/home/czarakas/coupled_PPE/code/utils/load_ensembles_postprocessing.py:67: FutureWarning: In xarray version 0.15 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  ds = xr.open_mfdataset(fpath)
/glade/u/home/czarakas/.conda/envs/EOF_env/lib/python3.7/site-packages/xarray/backends/api.py:941: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option to
open_mfdataset).
  from_openmfds=True,


In [8]:
if ensemble=='offline':
    ensemble_path='offline_simulations'
    refcase_id='OFFL0000_PI_v02'
elif ensemble=='coupled':
    ensemble_path='coupled_simulations'
    refcase_id='COUP0000_PI_SOM'
if domain=='atm':
    domain_suffix='.cam.h0.'
elif domain=='lnd':
    domain_suffix='.clm2.h0.'
    
basecase_lnd=xr.open_mfdataset('/glade/campaign/cgd/tss/people/czarakas/CoupledPPE/'+ensemble_path+'/'+
                                 refcase_id+'/'+domain+'/proc/tseries/month_1/'+refcase_id+domain_suffix+var+'.*.nc',
                              combine='by_coords')

### Check ensemble

In [9]:
ensemble_coupled[-1]=ensemble_coupled[-1].sel(time=slice('0049-02-01', '0189-01-01'))

In [10]:
expected_start_time=cftime.DatetimeNoLeap(49,2,1,0)
expected_end_time=cftime.DatetimeNoLeap(189,1,1,0)
for i,ds in enumerate(ensemble_coupled):
    #print(i)
    #print(ds.case_id)
    if np.size(ds.time)!=1680:
        print(ds.case_id)
        print('!!!!! Time size is wrong dimension')
    if ds.time.values[0] != expected_start_time:
        print(ds.time.values[0])
    if ds.time.values[-1] != expected_end_time:
        print(ds.time.values[-1])
    #print('----------')

In [12]:
expected_start_time=cftime.DatetimeNoLeap(49,1,16,12)
expected_end_time=cftime.DatetimeNoLeap(188,12,16,12)
for i,ds in enumerate(ensemble_coupled):
    #print(i)
    #print(ds.case_id)
    if np.size(ds.time)!=1680:
        #print(ds.case_id)
        print('!!!!! Time size is wrong dimension')
    if ds.time.values[0] != expected_start_time:
        print(ds.time.values[0])
    if ds.time.values[-1] != expected_end_time:
        print(ds.time.values[-1])
    #print('----------')

0049-02-01 00:00:00
0189-01-01 00:00:00
0049-02-01 00:00:00
0189-01-01 00:00:00
0049-02-01 00:00:00
0189-01-01 00:00:00
0049-02-01 00:00:00
0189-01-01 00:00:00
0049-02-01 00:00:00
0189-01-01 00:00:00
0049-02-01 00:00:00
0189-01-01 00:00:00
0049-02-01 00:00:00
0189-01-01 00:00:00
0049-02-01 00:00:00
0189-01-01 00:00:00
0049-02-01 00:00:00
0189-01-01 00:00:00
0049-02-01 00:00:00
0189-01-01 00:00:00
0049-02-01 00:00:00
0189-01-01 00:00:00
0049-02-01 00:00:00
0189-01-01 00:00:00
0049-02-01 00:00:00
0189-01-01 00:00:00
0049-02-01 00:00:00
0189-01-01 00:00:00
0049-02-01 00:00:00
0189-01-01 00:00:00
0049-02-01 00:00:00
0189-01-01 00:00:00
0049-02-01 00:00:00
0189-01-01 00:00:00
0049-02-01 00:00:00
0189-01-01 00:00:00
0049-02-01 00:00:00
0189-01-01 00:00:00
0049-02-01 00:00:00
0189-01-01 00:00:00
0049-02-01 00:00:00
0189-01-01 00:00:00
0049-02-01 00:00:00
0189-01-01 00:00:00
0049-02-01 00:00:00
0189-01-01 00:00:00
0049-02-01 00:00:00
0189-01-01 00:00:00
0049-02-01 00:00:00
0189-01-01 00:00:00


# Make datasets with all data

In [11]:
# Define grid for dataset
ds_grid = ensemble_coupled[0]

# Make data arrays
var_array_1time = make_multisimulation_dataset.make_empty_dataarray(ds_grid=ds_grid, var=var, keys=keys)

var_array=var_array_1time.expand_dims({"time": ds_grid.time},axis=0).copy()

/glade/u/home/czarakas/.conda/envs/EOF_env/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


In [12]:
# Put data in data arrays
for i, ds in enumerate(ensemble_coupled):
    print(i)
    var_array[:,:,:,i]=ds[var][:,:,:]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35


### Save the concatenated data

In [13]:
fname = var+'_timeseries_'+ensemble+'.nc'
ds_array=var_array.to_dataset(name=var)
ds_array.to_netcdf(path_out+fname)

# Calculate variables

## Precipitation

In [15]:
ds_rain = xr.open_dataset(path_out+'RAIN_FROM_ATM_timeseries_coupled.nc')
ds_snow = xr.open_dataset(path_out+'SNOW_FROM_ATM_timeseries_coupled.nc')

In [18]:
prec_from_atm = ds_rain.RAIN_FROM_ATM+ds_snow.SNOW_FROM_ATM

In [19]:
var='calculated_PREC_FROM_ATM'
fname = var+'_timeseries_'+ensemble+'.nc'
ds_array=prec_from_atm.to_dataset(name=var)
ds_array.to_netcdf(path_out+fname)

### Reference case

In [7]:
if ensemble=='offline':
    ensemble_path='offline_simulations'
    refcase_id='OFFL0000_PI_v02'
elif ensemble=='coupled':
    ensemble_path='coupled_simulations'
    refcase_id='COUP0000_PI_SOM'
if domain=='atm':
    domain_suffix='.cam.h0.'
elif domain=='lnd':
    domain_suffix='.clm2.h0.'

In [8]:
basecase_lnd_rain=xr.open_dataset('/glade/campaign/cgd/tss/czarakas/CoupledPPE/'+ensemble_path+'/'+
                                 refcase_id+'/'+domain+'/proc/tseries/'+refcase_id+domain_suffix+'timeseries.'+'RAIN_FROM_ATM'+'.nc')
basecase_lnd_snow=xr.open_dataset('/glade/campaign/cgd/tss/czarakas/CoupledPPE/'+ensemble_path+'/'+
                                 refcase_id+'/'+domain+'/proc/tseries/'+refcase_id+domain_suffix+'timeseries.'+'SNOW_FROM_ATM'+'.nc')

In [9]:
basecase_prec_from_atm = basecase_lnd_rain.RAIN_FROM_ATM+basecase_lnd_snow.SNOW_FROM_ATM

In [11]:
var='calculated_PREC_FROM_ATM'
ref_fpath='/glade/campaign/cgd/tss/czarakas/CoupledPPE/'+ensemble_path+'/'+refcase_id+'/'+domain+'/proc/tseries/'+refcase_id+domain_suffix+'timeseries.'+var+'.nc'


ds_array=basecase_prec_from_atm.to_dataset(name=var)
ds_array.to_netcdf(ref_fpath)